# Funcion completar tablas por mes

In [1]:
import pandas as pd
from IPython.display import clear_output

## Tablas en inglés

In [2]:
#cargando el CSV con la lista de departamentos y sus municipios
todos_municipios = pd.read_csv('resources/municipios_snis_corregido.csv')##tabla editada luego de eliminar muncipios repetidos
#cargando el indice
indice = pd.read_csv('resources/indice.csv',dtype={'grvar': str, 'subvar': str})#no olvidarse espeificar el type de datos que son grvar y subvar

In [3]:
todos_municipios = todos_municipios.rename(columns={'DEPARTAMENTO': 'DEPARTAMENT', 'COD_MUNICIPIO': 'code_municipality', 'MUNICIPIO': 'municipality'})
todos_municipios = todos_municipios[['DEPARTAMENT', 'code_municipality', 'municipality']]
todos_municipios

,DEPARTAMENT,code_municipality,municipality
0,CHUQUISACA,10101,SUCRE
1,CHUQUISACA,10102,YOTALA
2,CHUQUISACA,10103,POROMA
3,CHUQUISACA,10201,AZURDUY
4,CHUQUISACA,10202,TARVITA
...,...,...,...
337,PANDO,90401,NACEBE (SANTA ROSA DEL ABUNA)
338,PANDO,90402,INGAVI (HUMAITA)
339,PANDO,90501,NUEVO MANOA (NUEVA ESPERANZA)
340,PANDO,90502,VILLA NUEVA (LOMA ALTA)


In [4]:
indice

,year,filename,grvar,grupo,subvar,variable
0,2005,datos/2005_diagnostic-suspicion_whooping-cough...,01,diagnostic suspicion,03,whooping cough
1,2005,datos/2005_diagnostic-suspicion_diphtheria.csv,01,diagnostic suspicion,04,diphtheria
2,2005,datos/2005_diagnostic-suspicion_bolivian-hemor...,02,diagnostic suspicion,09,Bolivian hemorrhagic fever
3,2005,datos/2005_diagnostic-suspicion_hantavirus-dis...,02,diagnostic suspicion,14,hantavirus disease
4,2005,datos/2005_other-infections_acute-respiratory-...,06,other infections,02,acute respiratory infection without pneumonia
...,...,...,...,...,...,...
193,2021,datos/2021_other-infections_pneumonia.csv,06,other infections,12,pneumonia
194,2021,datos/2021_mortality_under-5-mortality-due-to-...,09,mortality,10,under 5 mortality due to pneumonia
195,2021,datos/2021_vector-borne-diseases-vbd_leishmani...,17,vector-borne diseases (VBD),01,leishmaniasis
196,2021,datos/2021_vector-borne-diseases-vbd_acute-cha...,17,vector-borne diseases (VBD),02,acute Chagas disease


In [5]:
def completando (indice_tab, municipios):
    errores = []
    len_seleccion = len(indice_tab)
    for i, f in enumerate(indice_tab.filename):
        clear_output(wait=True)
        print('{} / {}'.format(i+1, len_seleccion))
        try:
            dfi = pd.read_csv('{}'.format(f))
            #retiro los posibles duplicados por codigo
            dfi.drop_duplicates(subset=['code_municipality', 'month'])
            # Crea un DataFrame con todas las fechas únicas
            dfi_fechas=dfi[['month']].drop_duplicates()
            # Realiza una combinación (merge) entre los DataFrames de municipios y fechas
            df_com = pd.merge(municipios, dfi_fechas, how='cross')
            # Realiza una fusión (merge) con el DataFrame original para obtener los valores correspondientes
            
            dfi_com = pd.merge(df_com, dfi, on=['DEPARTAMENT','code_municipality','municipality', 'month'], how='left') 
            dfi_com['group'] = dfi['group'].unique()[0]  # Asigna el valor único de 'grupo' en tabla_incompleta
            dfi_com['variable'] = dfi['variable'].unique()[0]  # Asigna el valor único de 'variable' en tabla_incompleta           
            dfi_com.iloc[:, 6:] = dfi_com.iloc[:, 6:].fillna(0).astype(int)
            dfi_com.to_csv('completando/{}'.format(f.split('/')[1]), index=False)
        except Exception as e:
            errores.append({'filename': f, 'error': e})

    return errores


In [6]:
completamos = completando(indice, todos_municipios)

198 / 198


C:\Users\USER\AppData\Local\Temp\ipykernel_8332\1064766230.py:20: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfi_com.iloc[:, 6:] = dfi_com.iloc[:, 6:].fillna(0).astype(int)


In [7]:
completamos

[]

In [10]:
#aqui la tabla deberia salir completa 
indice_new = pd.read_csv('completando/indice_new.csv',dtype={'grvar': str, 'subvar': str})#no olvidarse espeificar el type de datos que son grvar y subvar

In [11]:
pd.read_csv('{}'.format(indice_new.sample()['filename'].iloc[0]))


,DEPARTAMENT,code_municipality,municipality,month,group,variable,under 1 year,1 to 4 years,5 to 9 years,10 to 20 years,21 to 59 years,60 and more years,total
0,CHUQUISACA,10101,SUCRE,2010-01-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
1,CHUQUISACA,10101,SUCRE,2010-02-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
2,CHUQUISACA,10101,SUCRE,2010-03-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
3,CHUQUISACA,10101,SUCRE,2010-04-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
4,CHUQUISACA,10101,SUCRE,2010-05-01,diagnostic suspicion,influenza pandemic,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,PANDO,90503,EUREKA (SANTOS MERCADO),2010-08-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
4100,PANDO,90503,EUREKA (SANTOS MERCADO),2010-09-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
4101,PANDO,90503,EUREKA (SANTOS MERCADO),2010-10-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0
4102,PANDO,90503,EUREKA (SANTOS MERCADO),2010-11-01,diagnostic suspicion,influenza pandemic,0,0,0,0,0,0,0


## Tablas anuales en español 

In [2]:
#cargando el CSV con la lista de departamentos y sus municipios
todos_municipios = pd.read_csv('resources/municipios_snis_corregido.csv')##tabla editada luego de eliminar muncipios repetidos
#cargando el indice
indice = pd.read_csv('resources/indice.csv',dtype={'grvar': str, 'subvar': str})#no olvidarse espeificar el type de datos que son grvar y subvar

In [5]:
todos_municipios = todos_municipios.rename(columns={'DEPARTAMENTO': 'DEPARTAMENTO', 'COD_MUNICIPIO': 'codigo_municipio', 'MUNICIPIO': 'municipio'})
todos_municipios = todos_municipios[['DEPARTAMENTO','codigo_municipio','municipio']]
todos_municipios

,DEPARTAMENTO,codigo_municipio,municipio
0,CHUQUISACA,10101,SUCRE
1,CHUQUISACA,10102,YOTALA
2,CHUQUISACA,10103,POROMA
3,CHUQUISACA,10201,AZURDUY
4,CHUQUISACA,10202,TARVITA
...,...,...,...
337,PANDO,90401,NACEBE (SANTA ROSA DEL ABUNA)
338,PANDO,90402,INGAVI (HUMAITA)
339,PANDO,90501,NUEVO MANOA (NUEVA ESPERANZA)
340,PANDO,90502,VILLA NUEVA (LOMA ALTA)


In [5]:
indice

,year,filename,grvar,grupo,subvar,variable
0,2005,datos/2005_sospecha-diagnostica_tos-ferina.csv,01,sospecha diagnostica,03,tos ferina
1,2005,datos/2005_sospecha-diagnostica_difteria.csv,01,sospecha diagnostica,04,difteria
2,2005,datos/2005_sospecha-diagnostica_fiebre-hemorra...,02,sospecha diagnostica,09,fiebre hemorrag. bol
3,2005,datos/2005_sospecha-diagnostica_enfer-por-hant...,02,sospecha diagnostica,14,enfer. por hanta virus
4,2005,datos/2005_otras-infecciones_ira-sin-neumonia.csv,06,otras infecciones,02,ira sin neumonia
...,...,...,...,...,...,...
193,2021,datos/2021_otras-infecciones_neumonia.csv,06,otras infecciones,12,neumonia
194,2021,datos/2021_mortalidad_muerte-menor-de-5-anos-p...,09,mortalidad,10,muerte menor de 5 anos por neumonia
195,2021,datos/2021_enfermedades-transmitidas-por-vecto...,17,enfermedades transmitidas por vectores (etv),01,leishmaniasis
196,2021,datos/2021_enfermedades-transmitidas-por-vecto...,17,enfermedades transmitidas por vectores (etv),02,chagas agudo


In [6]:
def completando2 (indice_tab, municipios):
    errores = []
    len_seleccion = len(indice_tab)
    for i, f in enumerate(indice_tab.filename):
        clear_output(wait=True)
        print('{} / {}'.format(i+1, len_seleccion))
        try:
            dfi = pd.read_csv('{}'.format(f))
            #retiro los posibles duplicados por codigo
            dfi.drop_duplicates(subset=['codigo_municipio', 'mes'])
            # Crea un DataFrame con todas las fechas únicas
            dfi_fechas=dfi[['mes']].drop_duplicates()
            # Realiza una combinación (merge) entre los DataFrames de municipios y fechas
            df_com = pd.merge(municipios, dfi_fechas, how='cross')
            # Realiza una fusión (merge) con el DataFrame original para obtener los valores correspondientes
            dfi_com = pd.merge(df_com, dfi, on=['DEPARTAMENTO','codigo_municipio','municipio','mes'], how='left')

            dfi_com['grupo'] = dfi['grupo'].unique()[0]  # Asigna el valor único de 'grupo' en tabla_incompleta
            dfi_com['variable'] = dfi['variable'].unique()[0]  # Asigna el valor único de 'variable' en tabla_incompleta
            dfi_com.iloc[:, 6:] = dfi_com.iloc[:, 6:].fillna(0).astype(int)
            dfi_com.to_csv('completando/{}'.format(f.split('/')[1]), index=False)
        except Exception as e:
            errores.append({'filename': f, 'error': e})

    return errores


In [7]:
completamos = completando2(indice, todos_municipios)

198 / 198


C:\Users\USER\AppData\Local\Temp\ipykernel_6428\209807417.py:20: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfi_com.iloc[:, 6:] = dfi_com.iloc[:, 6:].fillna(0).astype(int)


In [8]:
completamos

[]

In [3]:
#aqui la tabla deberia salir completa 
indice_nuevo = pd.read_csv('completando/indice_nuevo.csv',dtype={'grvar': str, 'subvar': str})#no olvidarse espeificar el type de datos que son grvar y subvar

In [4]:
pd.read_csv('{}'.format(indice_nuevo.sample()['filename'].iloc[0]))

,DEPARTAMENTO,codigo_municipio,municipio,mes,grupo,variable,menores a 1 año,de 1 a 4 años,de 5 a 9 años,de 10 a 20 años,de 21 a 59 años,de 60 y más años,total
0,CHUQUISACA,10101,SUCRE,2010-01-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
1,CHUQUISACA,10101,SUCRE,2010-02-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
2,CHUQUISACA,10101,SUCRE,2010-03-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
3,CHUQUISACA,10101,SUCRE,2010-04-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
4,CHUQUISACA,10101,SUCRE,2010-05-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,PANDO,90503,EUREKA (SANTOS MERCADO),2010-08-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
4100,PANDO,90503,EUREKA (SANTOS MERCADO),2010-09-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
4101,PANDO,90503,EUREKA (SANTOS MERCADO),2010-10-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
4102,PANDO,90503,EUREKA (SANTOS MERCADO),2010-11-01,sospecha diagnostica,influenza - pandemica,0,0,0,0,0,0,0
